In [1]:
import pandas as pd
import numpy as np
import pandas_profiling
from datetime import timedelta

In [2]:
!ls ./../data/

Data Description.pdf pings.csv
drivers.csv          test.csv


## Driver Data Analysis

In [3]:
df_drivers = pd.read_csv('./../data/drivers.csv')

In [4]:
print("Intially Number of Rows in Driver.csv", len(df_drivers))

Intially Number of Rows in Driver.csv 2500


In [5]:
df_drivers.head(10)

,driver_id,gender,age,number_of_kids
0,979863,MALE,26,2
1,780123,MALE,60,2
2,614848,MALE,45,4
3,775046,MALE,62,3
4,991601,MALE,23,0
5,670477,MALE,36,1
6,194053,MALE,35,0
7,364596,FEMALE,36,4
8,640428,MALE,28,0
9,154517,MALE,25,2


In [6]:
## Driver Profile Data contains Duplicates
df_drivers[df_drivers.duplicated(subset=['driver_id'],keep=False)]

,driver_id,gender,age,number_of_kids
231,320227,MALE,22,4
388,320227,MALE,42,4
1311,799250,FEMALE,27,4
1550,800600,FEMALE,22,0
1765,799250,FEMALE,23,0
2008,800600,FEMALE,28,1


### Same driver_id cannot have different profile data
#### Considering first occurence to be correct. 
#### We have 2497 unique driver_id: profiles

In [7]:
## Keeping first occurence
df_drivers.drop_duplicates(subset=['driver_id'], inplace = True)

In [8]:
df_drivers.head(10)

,driver_id,gender,age,number_of_kids
0,979863,MALE,26,2
1,780123,MALE,60,2
2,614848,MALE,45,4
3,775046,MALE,62,3
4,991601,MALE,23,0
5,670477,MALE,36,1
6,194053,MALE,35,0
7,364596,FEMALE,36,4
8,640428,MALE,28,0
9,154517,MALE,25,2


In [9]:
# Info of Driver Dataset
df_drivers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2497 entries, 0 to 2499
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   driver_id       2497 non-null   int64 
 1   gender          2497 non-null   object
 2   age             2497 non-null   int64 
 3   number_of_kids  2497 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 97.5+ KB


In [10]:
#Count missing values
np.count_nonzero(df_drivers.isnull().values)

0

In [11]:
df_drivers.reset_index(inplace = True, drop = True)

In [12]:
profile = df_drivers.profile_report(title='Driver DataSet Analysis')
profile.to_file(output_file="../data_analysis_EDA/driver_data.html")
df_drivers.to_csv('../processed_data_and_models/driver.csv',index = False)

## Pings Data Analysis and Preprocessing

In [13]:
df_pings = pd.read_csv('./../data/pings.csv')

In [14]:
print("Intially Number of Rows in pings.csv", len(df_pings))

Intially Number of Rows in pings.csv 50528701


In [15]:
df_pings.head(10)

,driver_id,ping_timestamp
0,899313,1496278800
1,373017,1496278800
2,798984,1496278800
3,245966,1496278800
4,689783,1496278800
5,830028,1496278800
6,734310,1496278800
7,788584,1496278800
8,899313,1496278815
9,798984,1496278815


In [16]:
df_pings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50528701 entries, 0 to 50528700
Data columns (total 2 columns):
 #   Column          Dtype
---  ------          -----
 0   driver_id       int64
 1   ping_timestamp  int64
dtypes: int64(2)
memory usage: 771.0 MB


In [17]:
df_pings.sort_values(by = ['driver_id', 'ping_timestamp']).reset_index(drop = True)

,driver_id,ping_timestamp
0,111556,1496279340
1,111556,1496279355
2,111556,1496279370
3,111556,1496279400
4,111556,1496279430
...,...,...
50528696,998229,1498091640
50528697,998229,1498091655
50528698,998229,1498091670
50528699,998229,1498091685


In [18]:
df_pings.head(10)

,driver_id,ping_timestamp
0,899313,1496278800
1,373017,1496278800
2,798984,1496278800
3,245966,1496278800
4,689783,1496278800
5,830028,1496278800
6,734310,1496278800
7,788584,1496278800
8,899313,1496278815
9,798984,1496278815


In [19]:
# Count missing values
np.count_nonzero(df_pings.isnull().values)

0

In [20]:
# Check Data contains duplicates 
len(df_pings[df_pings.duplicated(keep=False)])

79086

In [21]:
df_pings.drop_duplicates(subset= ['driver_id', 'ping_timestamp'], inplace = True)

In [22]:
df_pings['ping_date']= pd.to_datetime(df_pings['ping_timestamp'], unit= 's')

In [23]:
## Converting UTC Time to IST by adding 05:30 time difference
df_pings['ping_date'] = df_pings['ping_date']+pd.to_timedelta('05:30:00.00000')

In [24]:
df_pings['ping_date'] = df_pings['ping_date'].dt.date

## We know that: every 15 seconds indicating that driver is available.
## Methodology of Lag and Lead will be used here. 
### difference(t) = observed(t) - observed(t-1)
### If difference between two ping_timestamps is less than `ping_threshold` means driver was available for that time. 

### To get lag of interval =1, on grouped date by (driverid and ping_date) we can used shift() or diff()

In [25]:
# grouped difference on driverid and ping_date
df_pings['diff'] = df_pings.groupby(['driver_id', 'ping_date'])['ping_timestamp'].diff().fillna(0)

In [26]:
df_pings[(df_pings['driver_id']==111556)]

,driver_id,ping_timestamp,ping_date,diff
1315,111556,1496279340,2017-06-01,0.0
1388,111556,1496279355,2017-06-01,15.0
1453,111556,1496279370,2017-06-01,15.0
1593,111556,1496279400,2017-06-01,30.0
1749,111556,1496279430,2017-06-01,30.0
...,...,...,...,...
48602451,111556,1498020645,2017-06-21,15.0
48603510,111556,1498020660,2017-06-21,15.0
48604563,111556,1498020675,2017-06-21,15.0
48605616,111556,1498020690,2017-06-21,15.0


In [27]:
df_pings['diff'].value_counts().head(10)

15.0       45425180
30.0        4545394
45.0         425575
0.0           42331
60.0          41808
75.0           4014
90.0            457
105.0            45
56940.0          22
60480.0          18
Name: diff, dtype: int64

### To get hours of driver being available for a day we sum all difference timestamps which are within PING_THRESHOLD 

In [28]:
PING_THRESHOLD = 60

In [29]:
df_pings[df_pings['diff']<=PING_THRESHOLD].head()

,driver_id,ping_timestamp,ping_date,diff
0,899313,1496278800,2017-06-01,0.0
1,373017,1496278800,2017-06-01,0.0
2,798984,1496278800,2017-06-01,0.0
3,245966,1496278800,2017-06-01,0.0
4,689783,1496278800,2017-06-01,0.0


In [30]:
data = pd.DataFrame(df_pings[df_pings['diff']<=PING_THRESHOLD].groupby(by=['driver_id','ping_date'])['diff'].agg('sum')).reset_index()
data.columns = ['driver_id', 'date','available_hours']
# Secs to Hours
data['available_hours'] = round(data['available_hours']*(1.0/3600.0), 1)

In [31]:
data.head(8)

,driver_id,date,available_hours
0,111556,2017-06-01,2.2
1,111556,2017-06-02,2.5
2,111556,2017-06-05,4.7
3,111556,2017-06-06,3.1
4,111556,2017-06-07,2.6
5,111556,2017-06-08,3.2
6,111556,2017-06-09,4.4
7,111556,2017-06-12,3.7


In [32]:
## Fill in missing Date For Driver with Available_hours = 0
data.date = pd.to_datetime(data.date)
data = data.set_index(
    ['date', 'driver_id']
).unstack(
    fill_value=0
).asfreq(
    'D', fill_value=0
).stack().sort_index(level=1).reset_index()

In [33]:
data

,date,driver_id,available_hours
0,2017-06-01,111556,2.2
1,2017-06-02,111556,2.5
2,2017-06-03,111556,0.0
3,2017-06-04,111556,0.0
4,2017-06-05,111556,4.7
...,...,...,...
54555,2017-06-18,998229,5.3
54556,2017-06-19,998229,6.0
54557,2017-06-20,998229,6.0
54558,2017-06-21,998229,2.9


### DayofWeeks: where 0: Monday and 6: Sunday

In [34]:
data['dayofweek'] = data['date'].dt.dayofweek
data.head()

,date,driver_id,available_hours,dayofweek
0,2017-06-01,111556,2.2,3
1,2017-06-02,111556,2.5,4
2,2017-06-03,111556,0.0,5
3,2017-06-04,111556,0.0,6
4,2017-06-05,111556,4.7,0


### For some drivers there is ping data for even 22 June, removing that. 
#### Given in Problem Statement Data is provided from 1st June to 21st June

In [35]:
data= data[data.date!='2017-06-22']

In [36]:
data.reset_index(drop = True, inplace = True)

In [37]:
profile = data.profile_report(title='Driver_Pings Data Analysis')

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [38]:
profile.to_file(output_file="../data_analysis_EDA/driver_pings_hours.html")

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [39]:
data.to_csv('../processed_data_and_models/Pings_Grouped_Data.csv',index = False)

In [40]:
pd.read_csv('./../data/test.csv').profile_report(title = 'Drivers_Test Data').to_file(output_file="../data_analysis_EDA/test_driver.html")